In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from tqdm import tqdm
import numpy as np
from sklearn.metrics import confusion_matrix

from eye_dataset_v1 import *
from eye_model_v1 import *

# new iso l2 data collection pair eye - get softmax scores pair eye

In [59]:
from sklearn.metrics import confusion_matrix

mapping = {
    'open':0,
    'close':1,
    'block':2,
    'no_pred':3
}

reverse_mapping = {v:k for k,v in mapping.items()}

In [60]:
pairPath='~/data/aurora/ISO_L2_eye_mouth_blink/eye/data/v3.4/v3.4.0_test_pairs_full.csv'
pair_df = pd.read_csv(pairPath)
pair_df.columns

Index(['image_path_left', 'image_path_right', 'left_bbox', 'right_bbox',
       'left_label', 'right_label', 'pair_label', 'original_image_path',
       'glasses', 'original_label', 'split_type', 'original_bbox', 'source',
       'person_name', 'old_source_name', 'folder_source', 'collection_class',
       'collection_class_run', 'eye_score', 'collection_label', 'model_label',
       'label_description', 'specific_source', 'new_label', 'labeller',
       'left_v2.6.1_epoch908_softmax_score',
       'right_v2.6.1_epoch908_softmax_score',
       'left_v3.2.5_epoch90_softmax_score',
       'right_v3.2.5_epoch90_softmax_score',
       'left_v3.3.5_epoch816_softmax_score',
       'right_v3.3.5_epoch816_softmax_score',
       'left_v3.3.4.1_epoch987_softmax_score',
       'right_v3.3.4.1_epoch987_softmax_score'],
      dtype='object')

In [61]:
len(pair_df),pair_df.pair_label.value_counts()

(700,
 pair_label
 block    355
 close    173
 open     172
 Name: count, dtype: int64)

In [62]:
# preload pair eyes

def load_image_for_preload(input_param):
    expend_ratio = 0
    image = cv2.cvtColor(cv2.imread(input_param['image_path']), cv2.COLOR_BGR2RGB)
    l, t, r, b = input_param['bbox']
    w, h = r - l, b - t
    l, r = max(0, l - int(expend_ratio * w / 2)), min(image.shape[1] - 1, r + int(expend_ratio * w / 2))
    t, b = max(0, t - int(expend_ratio * h / 2)), min(image.shape[0] - 1, b + int(expend_ratio * h / 2))
    input_dim = input_param['input_dim']
    image = cv2.resize(image[t:b, l:r], (input_dim, input_dim))
    return image

def apply_input_transforms(image): #apply /255 and totensor for val and test, can apply for train after augmentations
    transforms=Compose([
        ToTensor()
    ])
    image=transforms(image)
    return image 

left_eyes,right_eyes=[],[]

for ind,row in tqdm(pair_df.iterrows()):
    image_path_left = row['image_path_left'] 
    image_path_right = row['image_path_right']
    left_bbox = row['left_bbox']
    right_bbox = row['right_bbox']
    left_label = row['left_label']
    right_label = row['right_label']
    
    left_params={'image_path':image_path_left,'bbox':eval(left_bbox),'input_dim':80}
    right_params={'image_path':image_path_right,'bbox':eval(right_bbox),'input_dim':80}
    
    left_eye_image = load_image_for_preload(left_params)
    right_eye_image = load_image_for_preload(right_params)    
    left_label = mapping[left_label]
    right_label = mapping[right_label]    
    left_eye_image = apply_input_transforms(left_eye_image)
    right_eye_image = apply_input_transforms(right_eye_image)
    
    left_eyes.append([left_eye_image,left_label])
    right_eyes.append([right_eye_image,right_label])
    
len(left_eyes),len(right_eyes)

700it [00:02, 342.86it/s]


(700, 700)

In [63]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
pair_df.columns

Index(['image_path_left', 'image_path_right', 'left_bbox', 'right_bbox',
       'left_label', 'right_label', 'pair_label', 'original_image_path',
       'glasses', 'original_label', 'split_type', 'original_bbox', 'source',
       'person_name', 'old_source_name', 'folder_source', 'collection_class',
       'collection_class_run', 'eye_score', 'collection_label', 'model_label',
       'label_description', 'specific_source', 'new_label', 'labeller',
       'left_v2.6.1_epoch908_softmax_score',
       'right_v2.6.1_epoch908_softmax_score',
       'left_v3.2.5_epoch90_softmax_score',
       'right_v3.2.5_epoch90_softmax_score',
       'left_v3.3.5_epoch816_softmax_score',
       'right_v3.3.5_epoch816_softmax_score',
       'left_v3.3.4.1_epoch987_softmax_score',
       'right_v3.3.4.1_epoch987_softmax_score'],
      dtype='object')

In [67]:
pairs=[['v2.6.1','908'],['v3.2.5','90'],['v3.3.5','816'],['v3.3.4.1','987']]

model_key = 'mobilenetv3_small_050'

for output_folder,epoch in pairs:
    model_path = f'/home/jovyan/data/aurora/ISO_L2_eye_mouth_blink/eye/train/result/{output_folder}/weights/epoch{epoch}.pt' 
    try: 
        # setup model - from scratch or load model for finetune
        model = Eye_Net(
            model_key = model_key,
            in_channel = 3
        ).to(device)
        model.load_state_dict(torch.load(os.path.join(model_path), map_location = device))
        _ = model.eval()
        print('model loaded succesfully')

        # start inference
        left_softmax_output_list,right_softmax_output_list = [],[]
        for i,((left_image,left_label),(right_image,right_label)) in enumerate(tqdm(zip(left_eyes,right_eyes))):

            left_logits = model(left_image.unsqueeze(0).cuda())
            left_logits = left_logits.cpu().detach()
            left_softmax_output = nn.Softmax(dim=1)(left_logits)
            left_softmax_output = [round(x,5) for x in left_softmax_output.numpy()[0].tolist()]
            left_softmax_output_list.append(left_softmax_output)
            
            
            right_logits = model(right_image.unsqueeze(0).cuda())
            right_logits = right_logits.cpu().detach()
            right_softmax_output = nn.Softmax(dim=1)(right_logits)
            right_softmax_output = [round(x,5) for x in right_softmax_output.numpy()[0].tolist()]
            right_softmax_output_list.append(right_softmax_output)

        # save softmax output to test_df
        left_model_softmax_col_name = f'left_{output_folder}_epoch{epoch}_softmax_score'
        pair_df[left_model_softmax_col_name]=left_softmax_output_list
        
        # save softmax output to test_df
        right_model_softmax_col_name = f'right_{output_folder}_epoch{epoch}_softmax_score'
        pair_df[right_model_softmax_col_name]=right_softmax_output_list
        
    except Exception as e:
        print(epoch,e)
        
# pair_df.head()

model loaded succesfully


700it [00:12, 55.70it/s]


model loaded succesfully


700it [00:10, 66.44it/s]


model loaded succesfully


700it [00:10, 66.18it/s]


model loaded succesfully


700it [00:11, 60.22it/s]


In [68]:
pair_df.to_csv(pairPath,index=False)

# pair eye prediction using threshold

In [69]:
pair_df = pd.read_csv(pairPath)
pair_df

,image_path_left,image_path_right,left_bbox,right_bbox,left_label,right_label,pair_label,original_image_path,glasses,original_label,...,new_label,labeller,left_v2.6.1_epoch908_softmax_score,right_v2.6.1_epoch908_softmax_score,left_v3.2.5_epoch90_softmax_score,right_v3.2.5_epoch90_softmax_score,left_v3.3.5_epoch816_softmax_score,right_v3.3.5_epoch816_softmax_score,left_v3.3.4.1_epoch987_softmax_score,right_v3.3.4.1_epoch987_softmax_score
0,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,"[0,0,80,80]","[0,0,80,80]",close,close,close,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,NaN,close,...,2.0,haidi,"[0.00225, 0.93446, 0.06329]","[5e-05, 0.06346, 0.93649]","[0.01366, 0.60081, 0.38553]","[0.01063, 0.58689, 0.40249]","[0.00326, 0.98626, 0.01048]","[0.00202, 0.2383, 0.75968]","[0.00099, 0.98542, 0.01359]","[0.00081, 0.60879, 0.3904]"
1,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,"[0,0,80,80]","[0,0,80,80]",open,open,open,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,NaN,open,...,1.0,haidi,"[0.99491, 7e-05, 0.00501]","[0.99428, 0.00017, 0.00554]","[0.5938, 0.13897, 0.26723]","[0.58498, 0.32836, 0.08666]","[0.98707, 0.0001, 0.01283]","[0.99697, 0.00137, 0.00166]","[0.99849, 0.00031, 0.00119]","[0.98467, 0.00299, 0.01235]"
2,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,"[0,0,80,80]","[0,0,80,80]",open,open,open,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,NaN,open,...,1.0,haidi,"[0.99962, 1e-05, 0.00037]","[0.99914, 0.0, 0.00085]","[0.80123, 0.05483, 0.14394]","[0.82647, 0.0892, 0.08433]","[0.9828, 2e-05, 0.01719]","[0.99746, 1e-05, 0.00253]","[0.99984, 1e-05, 0.00015]","[0.99775, 3e-05, 0.00222]"
3,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,"[0,0,80,80]","[0,0,80,80]",close,close,close,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,NaN,close,...,2.0,haidi,"[0.0, 1.0, 0.0]","[0.00033, 0.39601, 0.60366]","[0.00075, 0.81555, 0.1837]","[0.00057, 0.74479, 0.25464]","[0.0, 0.99987, 0.00013]","[0.00171, 0.34932, 0.64896]","[0.0, 0.99988, 0.00012]","[0.00028, 0.74582, 0.25389]"
4,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,"[0,0,80,80]","[0,0,80,80]",close,close,close,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,NaN,close,...,2.0,haidi,"[0.00217, 0.75424, 0.24359]","[0.00011, 0.62968, 0.37021]","[0.00522, 0.61655, 0.37823]","[0.03462, 0.36113, 0.60425]","[0.00882, 0.85572, 0.13547]","[0.00705, 0.76614, 0.22681]","[0.00124, 0.96564, 0.03312]","[0.00053, 0.90091, 0.09855]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,"[0,0,80,80]","[0,0,80,80]",block,block,block,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,NaN,block,...,5.0,haidi,"[0.32219, 5e-05, 0.67775]","[0.9925, 0.00015, 0.00736]","[0.15518, 0.00105, 0.84376]","[0.16301, 0.01687, 0.82012]","[0.04863, 2e-05, 0.95135]","[0.86383, 0.0006, 0.13557]","[1e-05, 0.0, 0.99999]","[0.18378, 1e-05, 0.81621]"
696,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,"[0,0,80,80]","[0,0,80,80]",block,block,block,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,NaN,block,...,5.0,haidi,"[0.72684, 4e-05, 0.27312]","[0.02932, 0.0001, 0.97058]","[0.02768, 0.00419, 0.96813]","[0.02978, 0.00353, 0.96669]","[0.00157, 4e-05, 0.9984]","[0.00179, 1e-05, 0.9982]","[0.0, 0.0, 1.0]","[3e-05, 0.0, 0.99996]"
697,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,"[0,0,80,80]","[0,0,80,80]",block,block,block,/home/jovyan/data/aurora/dataset/ISO_L2_data_c...,NaN,block,...,5.0,haidi,"[6e-05, 0.00014, 0.9998]","[0.00094, 0.00013, 0.99893]","[0.01019, 0.00867, 0.98114]","[0.01417, 0.003

In [70]:
pair_df.columns

Index(['image_path_left', 'image_path_right', 'left_bbox', 'right_bbox',
       'left_label', 'right_label', 'pair_label', 'original_image_path',
       'glasses', 'original_label', 'split_type', 'original_bbox', 'source',
       'person_name', 'old_source_name', 'folder_source', 'collection_class',
       'collection_class_run', 'eye_score', 'collection_label', 'model_label',
       'label_description', 'specific_source', 'new_label', 'labeller',
       'left_v2.6.1_epoch908_softmax_score',
       'right_v2.6.1_epoch908_softmax_score',
       'left_v3.2.5_epoch90_softmax_score',
       'right_v3.2.5_epoch90_softmax_score',
       'left_v3.3.5_epoch816_softmax_score',
       'right_v3.3.5_epoch816_softmax_score',
       'left_v3.3.4.1_epoch987_softmax_score',
       'right_v3.3.4.1_epoch987_softmax_score'],
      dtype='object')

In [72]:
from sklearn.metrics import confusion_matrix

mapping = {
    'open':0,
    'close':1,
    'block':2,
    'no_pred':3
}

reverse_mapping = {v:k for k,v in mapping.items()}

In [73]:
def get_pair_eye_prediction(left_score,right_score,open_score,close_thre,block_thre):
    if left_score[2] > block_thre or right_score[2] > block_thre:
        return 'block'
    if left_score[1] > close_thre and right_score[1] > close_thre:
        return 'close'
    if left_score[0] > open_thre and right_score[0] > open_thre:
        return 'open'
    else:
        return 'no_pred'

In [74]:
pair_df.columns[-10:]

Index(['new_label', 'labeller', 'left_v2.6.1_epoch908_softmax_score',
       'right_v2.6.1_epoch908_softmax_score',
       'left_v3.2.5_epoch90_softmax_score',
       'right_v3.2.5_epoch90_softmax_score',
       'left_v3.3.5_epoch816_softmax_score',
       'right_v3.3.5_epoch816_softmax_score',
       'left_v3.3.4.1_epoch987_softmax_score',
       'right_v3.3.4.1_epoch987_softmax_score'],
      dtype='object')

In [84]:
open_thre,close_thre,block_thre = 0.5,0.5,0.6

left_column = 'left_v2.6.1_epoch908_softmax_score'
right_column = 'right_v2.6.1_epoch908_softmax_score'

In [85]:
# get prediction on separate old vs new dataset

mapping = {
    'open':0,
    'close':1,
    'block':2,
    'no_pred':3
}

dataset='old'

left_scores = [eval(x) for x in pair_df[left_column].tolist()]
right_scores = [eval(x) for x in pair_df[right_column].tolist()]
# left_scores = [x for x in current[left_column].tolist()]
# right_scores = [x for x in current[right_column].tolist()]
pair_labels = [x for x in pair_df['pair_label'].tolist()]

pair_predictions=[]
for (left_score,right_score) in tqdm(zip(left_scores,right_scores)):
    pair_predictions.append(get_pair_eye_prediction(left_score,right_score,open_thre,close_thre,block_thre))

pair_labels = [mapping[x] for x in pair_labels]
pair_predictions = [mapping[x] for x in pair_predictions]

cm = confusion_matrix(pair_labels,pair_predictions)

print("NEW DATASET")
print(f'test acc-open: {round(cm[0,0]/sum(cm[0]) *100 , 2)}')
print(f'test acc-close: {round(cm[1,1]/sum(cm[1]) *100 , 2)}')
print(f'test acc-block: {round(cm[2,2]/sum(cm[2]) *100 , 2)}')
print(cm)

700it [00:00, 559134.03it/s]

NEW DATASET
test acc-open: 95.93
test acc-close: 64.16
test acc-block: 58.03
[[165   0   5   2]
 [  0 111  53   9]
 [121   8 206  20]
 [  0   0   0   0]]


In [93]:
open_thre,close_thre,block_thre = 0.5,0.5,0.6
output_folder = 'v2.6.1'
epoch = 908

left_column = f'left_{output_folder}_epoch{epoch}_softmax_score'
right_column = f'right_{output_folder}_epoch{epoch}_softmax_score'

left_scores = [eval(x) for x in pair_df[left_column].tolist()]
right_scores = [eval(x) for x in pair_df[right_column].tolist()]
pair_labels = [x for x in pair_df['pair_label'].tolist()]

pair_predictions=[]
for (left_score,right_score) in zip(left_scores,right_scores):
    pair_predictions.append(get_pair_eye_prediction(left_score,right_score,open_thre,close_thre,block_thre))

pair_labels = [mapping[x] for x in pair_labels]
pair_predictions = [mapping[x] for x in pair_predictions]

pred_open,pred_close,pred_block,pred_no_pred = pair_predictions.count(0), pair_predictions.count(1), pair_predictions.count(2), pair_predictions.count(3)
print('predicted open close block no_pred')
print(pred_open,pred_close,pred_block,pred_no_pred)
print(f'block-acc: {round(pred_block/len(pair_labels)*100,2)}')

predicted open close block no_pred
121 8 206 20
block-acc: 58.03


In [144]:
open_thre,close_thre,block_thre = 0.5,0.4,0.4
output_folder = 'v3.3.4.1'
epoch = 987

left_column = f'left_{output_folder}_epoch{epoch}_softmax_score'
right_column = f'right_{output_folder}_epoch{epoch}_softmax_score'

left_scores = [eval(x) for x in pair_df[left_column].tolist()]
right_scores = [eval(x) for x in pair_df[right_column].tolist()]
pair_labels = [x for x in pair_df['pair_label'].tolist()]

pair_predictions=[]
for (left_score,right_score) in zip(left_scores,right_scores):
    pair_predictions.append(get_pair_eye_prediction(left_score,right_score,open_thre,close_thre,block_thre))

pair_labels = [mapping[x] for x in pair_labels]
pair_predictions = [mapping[x] for x in pair_predictions]

pred_open,pred_close,pred_block,pred_no_pred = pair_predictions.count(0), pair_predictions.count(1), pair_predictions.count(2), pair_predictions.count(3)
print('predicted open close block no_pred')
print(pred_open,pred_close,pred_block,pred_no_pred)
print(f'block-acc: {round(pred_block/len(pair_labels)*100,2)}')

predicted open close block no_pred
10 0 344 1
block-acc: 96.9


# case study

In [139]:
def load_image(image_path, bbox, resize_dim):
    expand_ratio = 0
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    l, t, r, b = bbox
    w, h = r - l, b - t
    l, r = max(0, l - int(expand_ratio * w / 2)), min(image.shape[1] - 1, r + int(expand_ratio * w / 2))
    t, b = max(0, t - int(expand_ratio * h / 2)), min(image.shape[0] - 1, b + int(expand_ratio * h / 2))
    image = cv2.resize(image[t:b, l:r], (resize_dim, resize_dim))
    return image

def apply_input_transforms(image): #apply /255 and totensor for val and test, can apply for train after augmentations
    transforms=Compose([
        ToTensor()
    ])
    image=transforms(image)
    return image 

def plot_pair_images_grid(df, num_images, n_cols, columns=[],resize_dim=0):
    indexes = df.index[:num_images]
    n_rows = int(np.ceil(len(indexes) / n_cols))
    
    for i, ind in enumerate(tqdm(indexes)):
        row = df.loc[ind]
        image_path_left = row['image_path_left']
        image_path_right = row['image_path_right']
        left_bbox = eval(row['left_bbox'])
        right_bbox = eval(row['right_bbox'])
        
        left_image = load_image(image_path=image_path_left, bbox=left_bbox, resize_dim=resize_dim)
        right_image = load_image(image_path=image_path_right, bbox=right_bbox, resize_dim=resize_dim)
        
        if columns:
            left_score_title,right_score_title='',''
            for column in columns:
                left_score_title+='\n'
                right_score_title+='\n'
                output_folder,epoch = column
                left_model_softmax_col_name = f'left_{output_folder}_epoch{epoch}_softmax_score'
                right_model_softmax_col_name = f'right_{output_folder}_epoch{epoch}_softmax_score'
                left_score=row[left_model_softmax_col_name]
                right_score=row[right_model_softmax_col_name]
                left_score = eval(left_score)
                right_score = eval(right_score)
                left_score=[round(x,2) for x in left_score]
                right_score=[round(x,2) for x in right_score]
                left_score_title+=f"{output_folder}: {str(left_score)}"
                right_score_title+=f"{output_folder}: {str(right_score)}"
        
        plt.subplot(n_rows, 2 * n_cols, 2 * i + 1)
        plt.imshow(left_image)
        plt.axis('off')
        plt.title(f'{ind} - left{left_score_title}',fontsize=7)

        plt.subplot(n_rows, 2 * n_cols, 2 * i + 2)
        plt.imshow(right_image)
        plt.axis('off')
        plt.title(f'{ind} - right{right_score_title}',fontsize=7)

    plt.subplots_adjust(hspace=0.1, wspace=0.05)
    plt.gcf().set_size_inches(3.5 * n_cols, 2.4 * n_rows)
    plt.show()

In [140]:
current = pair_df.iloc[:20]

In [141]:
current.columns

Index(['image_path_left', 'image_path_right', 'left_bbox', 'right_bbox',
       'left_label', 'right_label', 'pair_label', 'original_image_path',
       'glasses', 'original_label', 'split_type', 'original_bbox', 'source',
       'person_name', 'old_source_name', 'folder_source', 'collection_class',
       'collection_class_run', 'eye_score', 'collection_label', 'model_label',
       'label_description', 'specific_source', 'new_label', 'labeller',
       'left_v2.6.1_epoch908_softmax_score',
       'right_v2.6.1_epoch908_softmax_score',
       'left_v3.2.5_epoch90_softmax_score',
       'right_v3.2.5_epoch90_softmax_score',
       'left_v3.3.5_epoch816_softmax_score',
       'right_v3.3.5_epoch816_softmax_score',
       'left_v3.3.4.1_epoch987_softmax_score',
       'right_v3.3.4.1_epoch987_softmax_score'],
      dtype='object')

In [1]:
plot_pair_images_grid(current, num_images=20, n_cols=6, columns=[['v2.6.1','908'],['v3.3.4.1','987']],resize_dim=80)

NameError: name 'plot_pair_images_grid' is not defined